In [1]:
# Transformers installation
! pip install transformers datasets evaluate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 21.8 MB/s 
     |████████████████████████████████| 451 kB 65.2 MB/s 
     |████████████████████████████████| 81 kB 6.8 MB/s 
     |████████████████████████████████| 7.6 MB 65.6 MB/s 
     |████████████████████████████████| 182 kB 70.2 MB/s 
     |████████████████████████████████| 132 kB 72.7 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 127 kB 76.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from datasets import load_dataset

#use the data, 
# 1- having the header
# 2- having the change the -1 negitive  label to 2 lable.

#dataset = load_dataset('csv', data_files='/content/train.csv')
#dataset["train"][100]

In [3]:
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("/content/", data_files=data_files)

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/content-610fad7d23948e0a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
#task='sentiment'
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
#MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
MODEL

'cardiffnlp/twitter-roberta-base-sentiment-latest'

In [5]:
from transformers import AutoTokenizer

max_length = 512


tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=max_length)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)#.select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)#.select(range(1000))

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
#https://discuss.huggingface.co/t/combining-metrics-for-multiclass-predictions-evaluations/21792/11

import numpy as np
from datasets import load_metric

#accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric =  evaluate.load("precision")


def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    #results.update(accuracy_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(precision_metric.compute(predictions=preds, references = labels, average="micro"))
    return results

In [10]:
class ConfiguredMetric:
    def __init__(self, metric, *metric_args, **metric_kwargs):
        self.metric = metric
        self.metric_args = metric_args
        self.metric_kwargs = metric_kwargs
    
    def add(self, *args, **kwargs):
        return self.metric.add(*args, **kwargs)
    
    def add_batch(self, *args, **kwargs):
        return self.metric.add_batch(*args, **kwargs)

    def compute(self, *args, **kwargs):
        return self.metric.compute(*args, *self.metric_args, **kwargs, **self.metric_kwargs)

    @property
    def name(self):
        return self.metric.name

    def _feature_names(self):
        return self.metric._feature_names()

In [11]:
evaluate.combine([
    evaluate.load('accuracy'), 
    ConfiguredMetric(evaluate.load('f1'), average='macro'),
    ConfiguredMetric(evaluate.load('recall'), average='macro'),
    ConfiguredMetric(evaluate.load('precision'), average='macro')
])

In [12]:
from transformers import TrainingArguments, Trainer


repo_name = "Almetrix-twitter-roberta-base-sentiment"


training_args = TrainingArguments(
    
    output_dir= repo_name,          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
 
                                  )

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 733
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 460
  Number of trainable parameters = 124647939


Step,Training Loss
10,1.896800
20,1.551000
30,1.492400
40,1.247100
50,1.326100
60,1.093000
70,1.031600
80,1.017100
90,0.766800
100,0.742000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=460, training_loss=0.6147974519625954, metrics={'train_runtime': 337.7562, 'train_samples_per_second': 10.851, 'train_steps_per_second': 1.362, 'total_flos': 964310675973120.0, 'train_loss': 0.6147974519625954, 'epoch': 5.0})

In [15]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 315
  Batch size = 16


{'eval_loss': 1.196197748184204,
 'eval_f1': 0.6984126984126984,
 'eval_recall': 0.6984126984126984,
 'eval_precision': 0.6984126984126984,
 'eval_runtime': 9.1837,
 'eval_samples_per_second': 34.3,
 'eval_steps_per_second': 2.178,
 'epoch': 5.0}

In [16]:
#trainer.push_to_hub()